In [6]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir='/scratch/xyin/tmp')
bert_model = BertModel.from_pretrained('bert-base-uncased', cache_dir='/scratch/xyin/tmp')
text = "Hello ETH Zurich"
encoded_input = tokenizer(text, return_tensors='pt')
output = bert_model(**encoded_input)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
import numpy as np

bert_param = {}
for name, param in bert_model.named_parameters():
    print(name, ':', param.size())
    bert_param[name] = param.detach().numpy()

embeddings.word_embeddings.weight : torch.Size([30522, 768])
embeddings.position_embeddings.weight : torch.Size([512, 768])
embeddings.token_type_embeddings.weight : torch.Size([2, 768])
embeddings.LayerNorm.weight : torch.Size([768])
embeddings.LayerNorm.bias : torch.Size([768])
encoder.layer.0.attention.self.query.weight : torch.Size([768, 768])
encoder.layer.0.attention.self.query.bias : torch.Size([768])
encoder.layer.0.attention.self.key.weight : torch.Size([768, 768])
encoder.layer.0.attention.self.key.bias : torch.Size([768])
encoder.layer.0.attention.self.value.weight : torch.Size([768, 768])
encoder.layer.0.attention.self.value.bias : torch.Size([768])
encoder.layer.0.attention.output.dense.weight : torch.Size([768, 768])
encoder.layer.0.attention.output.dense.bias : torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.weight : torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.bias : torch.Size([768])
encoder.layer.0.intermediate.dense.weight : torch.Size([3

In [29]:
param = {}
param['wq'] = bert_param['encoder.layer.0.attention.self.query.weight']
param['q_bias'] = bert_param['encoder.layer.0.attention.self.query.bias']
param['wk'] = bert_param['encoder.layer.0.attention.self.key.weight']
param['k_bias'] = bert_param['encoder.layer.0.attention.self.key.bias']
param['wv'] = bert_param['encoder.layer.0.attention.self.value.weight']
param['v_bias'] = bert_param['encoder.layer.0.attention.self.value.bias']
param['wo'] = bert_param['encoder.layer.0.attention.output.dense.weight']
param['o_bias'] = bert_param['encoder.layer.0.attention.output.dense.bias']
param['alpha_1'] = bert_param['encoder.layer.0.attention.output.LayerNorm.weight']
param['beta_1'] = bert_param['encoder.layer.0.attention.output.LayerNorm.bias']
param['w1'] = bert_param['encoder.layer.0.intermediate.dense.weight']
param['bias_1'] = bert_param['encoder.layer.0.intermediate.dense.bias']
param['w2'] = bert_param['encoder.layer.0.output.dense.weight']
param['bias_2'] = bert_param['encoder.layer.0.output.dense.bias']
param['alpha_2'] = bert_param['encoder.layer.0.output.LayerNorm.weight']
param['beta_2'] = bert_param['encoder.layer.0.output.LayerNorm.bias']

    

In [35]:
def emit(name, array, f, alignment='NR_LANES*32'):
    f.write(".global %s\n" % name)
    f.write(".balign " + alignment + "\n")
    f.write("%s:\n" % name)
    bs = array.tobytes()
    for i in range(0, len(bs), 4):
        s = ""
        for n in range(4):
            s += "%02x" % bs[i+3-n]
        f.write("    .word 0x%s\n" % s)


In [38]:
f = open('/scratch/xyin/ara/apps/transformer/data.S', mode='w+')
f.write('.section .data, \"aw\",@progbits\n')
for name in param:
    emit(name, param[name], f)
f.close()